In [0]:
from pyspark.sql import SparkSession, functions as f

# Reading Hospital A departments data 
df_hosa = spark.read.parquet("/mnt/bronze/hosa/providers")

# Reading Hospital B departments data 
df_hosb = spark.read.parquet("/mnt/bronze/hosb/providers")

# Union two departments dataframes
df_merged = df_hosa.unionByName(df_hosb)
display(df_merged)

df_merged.createOrReplaceTempView("providers")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS rcm_hc_adb_ws.silver.providers (
  ProviderID STRING,
  FirstName STRING,
  LastName STRING,
  Specialization STRING,
  DeptID STRING,
  NPI LONG,
  datasource STRING,
  is_quarantined BOOLEAN
)
USING DELTA;


In [0]:
spark.sql("TRUNCATE TABLE rcm_hc_adb_ws.silver.providers")


In [0]:
%sql
INSERT INTO rcm_hc_adb_ws.silver.providers
SELECT DISTINCT
  ProviderID,
  FirstName,
  LastName,
  Specialization,
  DeptID,
  CAST(NPI AS INT) AS NPI,
  datasource,
  CASE 
    WHEN ProviderID IS NULL OR DeptID IS NULL THEN TRUE
    ELSE FALSE
  END AS is_quarantined
FROM providers;
